# initialize the Modules

src : https://towardsdatascience.com/10-minutes-to-building-a-binary-image-classifier-by-applying-transfer-learning-to-mobilenet-eab5a8719525 ; https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2/MobileNetV2 ; https://www.youtube.com/watch?v=w8Qx40tHeEM&ab_channel=SebastiaanMath%C3%B4t ; https://colab.research.google.com/drive/1XYAXOHiXNWqmKedCj1WRcJ6mGpr4aVC0?usp=sharing

In [38]:
!pip3 install keras keras_applications keras_preprocessing Keras-Applications sklearn

In [67]:
#store working directory
!pwd
!workingDir=$(pwd)

/home/albertstarfield/Documents/FileSekolah13(TE)/bangkit_error/runtime/Motionful-Project-Bangkit2022/src/MachineLearning/model/transferLearningRetrofitting_MobileNet


In [39]:
import keras

In [40]:
keras.__version__

'2.9.0'

## Initialize the Machine Learning Modules Tensorflow

In [41]:
import tensorflow as tf
import keras.applications.mobilenet_v2 as MN2
import numpy as np
from imageio import imread

In [42]:

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNetV2 as MN2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

### Initiate the model of MN2 or mobilenet

In [47]:
model = Sequential()

### The Layers

In [48]:
model.add(MN2(include_top = False, weights="imagenet", input_shape=(200, 200, 3)))
#200, 200, 3 depending on the original model
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Dense(1, activation = 'sigmoid'))
# set layers 0 to be untrainable
model.layers[0].trainable = False

In [49]:
#after adding the layers of cheese combine it by compile it
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.01), loss = 'binary_crossentropy', metrics = 'accuracy')

/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


# Retrofitting Training Data

> We are going to use yoga_poses dataset that is originated from the tensorflow   !wget -O yoga_poses.zip http://download.tensorflow.org/data/pose_classification/yoga_poses.zip
which includes 5 pose

1. Chair
2. Cobra
3. Dog
4. Tree
5. Warrior

In [64]:
!pwd; apt install wget -y; rm -f yoga_poses.zip ; rm -rf train test; wget http://download.tensorflow.org/data/pose_classification/yoga_poses.zip; rm -rf trainingDataset; mkdir trainingDataset; mv yoga_poses.zip trainingDataset

/home/albertstarfield/Documents/FileSekolah13(TE)/bangkit_error/runtime/Motionful-Project-Bangkit2022/src/MachineLearning/model/transferLearningRetrofitting_MobileNet
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
--2022-06-11 14:18:09--  http://download.tensorflow.org/data/pose_classification/yoga_poses.zip
Resolving download.tensorflow.org (download.tensorflow.org)... 34.101.5.48, 2404:f340:4010:1801::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|34.101.5.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102517581 (98M) [application/zip]
Saving to: ‘yoga_poses.zip’

yoga_poses.zip      100%[===================>]  97.77M  1.16MB/s    in 66s     

2022-06-11 14:19:15 (1.48 MB/s) - ‘yoga_poses.zip’ saved [102517581/102517581]



In [65]:
!pwd; ls; apt install zip -y; cd trainingDataset; unzip yoga_poses.zip

/home/albertstarfield/Documents/FileSekolah13(TE)/bangkit_error/runtime/Motionful-Project-Bangkit2022/src/MachineLearning/model/transferLearningRetrofitting_MobileNet
LICENSE			mobilenet_v1_1.0_224_quant.tflite
TrainMobileNetv2.ipynb	trainingDataset
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Archive:  yoga_poses.zip
  inflating: LICENSE                 
   creating: test/
   creating: test/cobra/
  inflating: test/cobra/guy3_cobra105.jpg  
  inflating: test/cobra/guy3_cobra058.jpg  
  inflating: test/cobra/guy3_cobra117.jpg  
  inflating: test/cobra/guy3_cobra118.jpg  
  inflating: test/cobra/guy3_cobra084.jpg  
  inflating: test/cobra/guy3_cobra078.jpg  
  inflating: test/cobra/guy3_cobra027.jpg  
  inflating: test/cobra/guy3_cobra056.jpg  
  inflating: test/cobra/guy3_cobra120.jpg  
  inflating: test/cobra/guy3_cobra090

  inflating: test/chair/guy3_chair115.jpg  
  inflating: test/chair/guy3_chair101.jpg  
  inflating: test/chair/guy3_chair142.jpg  
  inflating: test/chair/guy3_chair145.jpg  
  inflating: test/chair/guy3_chair095.jpg  
  inflating: test/chair/guy3_chair094.jpg  
  inflating: test/chair/guy3_chair140.jpg  
  inflating: test/chair/guy3_chair076.jpg  
  inflating: test/chair/guy3_chair126.jpg  
  inflating: test/chair/guy3_chair146.jpg  
  inflating: test/chair/guy3_chair105.jpg  
   creating: test/tree/
  inflating: test/tree/guy3_tree095.jpg  
  inflating: test/tree/guy3_tree139.jpg  
  inflating: test/tree/guy3_tree096.jpg  
  inflating: test/tree/guy3_tree159.jpg  
  inflating: test/tree/guy3_tree086.jpg  
  inflating: test/tree/guy3_tree115.jpg  
  inflating: test/tree/guy3_tree140.jpg  
  inflating: test/tree/guy3_tree105.jpg  
  inflating: test/tree/guy3_tree165.jpg  
  inflating: test/tree/guy3_tree167.jpg  
  inflating: test/tree/guy3_tree174.jpg  
  inflating: test/tree/guy3_tr

  inflating: test/dog/guy3_dog099.jpg  
  inflating: test/dog/guy3_dog048.jpg  
  inflating: test/dog/guy3_dog102.jpg  
  inflating: test/dog/guy3_dog061.jpg  
  inflating: test/dog/guy3_dog098.jpg  
  inflating: test/dog/guy3_dog058.jpg  
  inflating: test/dog/guy3_dog071.jpg  
  inflating: test/dog/guy3_dog072.jpg  
  inflating: test/dog/guy3_dog085.jpg  
  inflating: test/dog/guy3_dog044.jpg  
  inflating: test/dog/guy3_dog027.jpg  
  inflating: test/dog/guy3_dog100.jpg  
  inflating: test/dog/guy3_dog055.jpg  
  inflating: test/dog/guy3_dog065.jpg  
  inflating: test/dog/guy3_dog059.jpg  
  inflating: test/dog/guy3_dog056.jpg  
  inflating: test/dog/guy3_dog032.jpg  
  inflating: test/dog/guy3_dog079.jpg  
  inflating: test/dog/guy3_dog077.jpg  
  inflating: test/dog/guy3_dog083.jpg  
  inflating: test/dog/guy3_dog054.jpg  
  inflating: test/dog/guy3_dog050.jpg  
  inflating: test/dog/guy3_dog096.jpg  
  inflating: test/dog/guy3_dog080.jpg  
  inflating: test/dog/guy3_dog040.jpg  


  inflating: train/chair/guy_chair131.jpg  
  inflating: train/chair/girl2_chair073.jpg  
  inflating: train/chair/guy2_chair143.jpg  
  inflating: train/chair/girl2_chair120.jpg  
  inflating: train/chair/girl3_chair104.jpg  
  inflating: train/chair/girl1_chair135.jpg  
  inflating: train/chair/guy_chair085.jpg  
  inflating: train/chair/girl3_chair096.jpg  
  inflating: train/chair/girl1_chair151.jpg  
  inflating: train/chair/guy2_chair104.jpg  
  inflating: train/chair/girl3_chair091.jpg  
  inflating: train/chair/guy2_chair109.jpg  
  inflating: train/chair/guy2_chair137.jpg  
  inflating: train/chair/guy2_chair074.jpg  
  inflating: train/chair/guy_chair105.jpg  
  inflating: train/chair/girl2_chair148.jpg  
  inflating: train/chair/guy_chair090.jpg  
  inflating: train/chair/girl1_chair124.jpg  
  inflating: train/chair/girl2_chair114.jpg  
  inflating: train/chair/guy2_chair094.jpg  
  inflating: train/chair/girl3_chair076.jpg  
  inflating: train/chair/guy2_chair088.jpg  
  i

  inflating: train/tree/guy1_tree132.jpg  
  inflating: train/tree/guy2_tree138.jpg  
  inflating: train/tree/guy2_tree122.jpg  
  inflating: train/tree/girl3_tree115.jpg  
  inflating: train/tree/girl2_tree104.jpg  
  inflating: train/tree/girl1_tree088.jpg  
  inflating: train/tree/guy2_tree128.jpg  
  inflating: train/tree/girl2_tree163.jpg  
  inflating: train/tree/guy2_tree126.jpg  
  inflating: train/tree/guy1_tree123.jpg  
  inflating: train/tree/girl3_tree128.jpg  
  inflating: train/tree/girl2_tree133.jpg  
  inflating: train/tree/guy1_tree088.jpg  
  inflating: train/tree/girl2_tree134.jpg  
  inflating: train/tree/girl2_tree128.jpg  
  inflating: train/tree/guy2_tree125.jpg  
  inflating: train/tree/girl2_tree091.jpg  
  inflating: train/tree/girl1_tree119.jpg  
  inflating: train/tree/girl1_tree089.jpg  
  inflating: train/tree/guy1_tree149.jpg  
  inflating: train/tree/guy2_tree092.jpg  
  inflating: train/tree/girl1_tree137.jpg  
  inflating: train/tree/girl3_tree121.jpg 

  inflating: train/warrior/guy1_warrior125.jpg  
  inflating: train/warrior/guy1_warrior105.jpg  
  inflating: train/warrior/girl1_warrior075.jpg  
  inflating: train/warrior/girl1_warrior053.jpg  
  inflating: train/warrior/girl3_warrior129.jpg  
  inflating: train/warrior/girl3_warrior127.jpg  
  inflating: train/warrior/girl2_warrior083.jpg  
  inflating: train/warrior/girl2_warrior096.jpg  
  inflating: train/warrior/girl2_warrior052.jpg  
  inflating: train/warrior/girl3_warrior134.jpg  
  inflating: train/warrior/girl1_warrior143.jpg  
  inflating: train/warrior/guy2_warrior044.jpg  
  inflating: train/warrior/guy1_warrior124.jpg  
  inflating: train/warrior/guy2_warrior108.jpg  
  inflating: train/warrior/guy2_warrior079.jpg  
  inflating: train/warrior/girl2_warrior142.jpg  
  inflating: train/warrior/guy1_warrior145.jpg  
  inflating: train/warrior/girl3_warrior054.jpg  
  inflating: train/warrior/guy1_warrior073.jpg  
  inflating: train/warrior/girl2_warrior057.jpg  
  inflat

  inflating: train/dog/guy1_dog102.jpg  
  inflating: train/dog/guy1_dog049.jpg  
  inflating: train/dog/girl3_dog025.jpg  
  inflating: train/dog/guy1_dog061.jpg  
  inflating: train/dog/guy1_dog046.jpg  
  inflating: train/dog/guy1_dog081.jpg  
  inflating: train/dog/girl1_dog069.jpg  
  inflating: train/dog/guy2_dog058.jpg  
  inflating: train/dog/guy1_dog034.jpg  
  inflating: train/dog/girl2_dog087.jpg  
  inflating: train/dog/girl2_dog053.jpg  
  inflating: train/dog/guy1_dog093.jpg  
  inflating: train/dog/girl2_dog085.jpg  
  inflating: train/dog/girl1_dog094.jpg  
  inflating: train/dog/girl1_dog082.jpg  
  inflating: train/dog/guy2_dog098.jpg  
  inflating: train/dog/girl2_dog111.jpg  
  inflating: train/dog/girl1_dog052.jpg  
  inflating: train/dog/guy1_dog112.jpg  
  inflating: train/dog/guy1_dog105.jpg  
  inflating: train/dog/girl3_dog072.jpg  
  inflating: train/dog/guy2_dog106.jpg  
  inflating: train/dog/guy1_dog088.jpg  
  inflating: train/dog/guy1_dog064.jpg  
  infl

In [ ]:
!cd ${workingDir} 

# Train time!

In [ ]:
history = model.fit(train_generator,
steps_per_epoch=8,
epochs=15,
verbose=1,
validation_data = validation_generator,
validation_steps=8)